### 練習問題2.69
以下の手続きは、引数として記号・頻度ペアのリ スト(同じ記号が⼆つ以上のペアに出てくることはない)を取り、  
ハフマンアルゴリズムに従ってハフマン符号化木を⽣成する。

    (define (generate-huffman-tree pairs)
        (successive-merge (make-leaf-set pairs)))

make-leaf-setは、上で記述した、ペアのリストを葉の順序つき集合に変換する⼿続きである。  
successive-mergeは、集合の中で重みが最小の要素をmake-code-treeを使って順番にくっつけていき、  
最後に要素がひとつだけ残るようにするというものである。  
その要素が求めるハフマン木となる。  
この手続きを書け  
(この手続きにはちょっと厄介なところがあるが、  
 そこまで複雑ではない。もし⼿続きの設計が複雑になったとしたら、  
 ほぼ確実に何かを間違えている。  
 順序つきの集合表現を使っているということが大きな助けになる)

In [111]:
(define (make-leaf symbol weight) (list 'leaf symbol weight))
(define (leaf? object) (eq? (car object) 'leaf))
(define (symbol-leaf x) (cadr x))
(define (weight-leaf x) (caddr x))

; コンストラクタ
(define (make-code-tree left right)
  (list left right
        (append (symbols left) (symbols right))
        (+ (weight left) (weight right))
    )
  )

; セレクタ
(define (left-branch tree) (car tree))
(define (right-branch tree) (cadr tree))
(define (symbols tree)
  (if (leaf? tree) (list (symbol-leaf tree))
      (caddr tree)
    )
  )
(define (weight tree)
  (if (leaf? tree) (weight-leaf tree)
      (cadddr tree)
    )
  )

(define (decode bits tree)
  (define (decode-1 bits current-branch)
    (if (null? bits) '()
        (let ((next-branch (choose-branch (car bits) current-branch)))
          (if (leaf? next-branch) (cons (symbol-leaf next-branch) (decode-1 (cdr bits) tree))
              (decode-1 (cdr bits) next-branch))
          )
    )
    )
  (decode-1 bits tree)
)  
(define (choose-branch bit branch)
  (cond ((= bit 0) (left-branch branch))
        ((= bit 1) (right-branch branch))
        (else (error "bad bit: CHOOSE-BRANCH" bit))
        )
  )

(define (adjoin-set x set)
  (cond ((null? set) (list x))
        ((< (weight x) (weight (car set))) (cons x set))
        (else (cons (car set) (adjoin-set x (cdr set))))))

(define (make-leaf-set pairs)
  (if (null? pairs) '()
      (let ((pair (car pairs)))
        (adjoin-set (make-leaf (car pair) ; symbol
                               (cadr pair)) ; weight
                    (make-leaf-set (cdr pairs)))
        )
    )
  )

In [112]:
(define sample-tree
    (make-code-tree (make-leaf 'A 4)
        (make-code-tree (make-leaf 'B 2)
            (make-code-tree (make-leaf 'D 1)
                (make-leaf 'C 1)
            )
        )
    )
)
(define sample-message '(0 1 1 0 0 1 0 1 0 1 1 1 0))

In [113]:
; 復号
(decode sample-message sample-tree)

(A D A B B C A)

In [114]:
; ハフマン符号化木の確認
sample-tree

((leaf A 4) ((leaf B 2) ((leaf D 1) (leaf C 1) (D C) 2) (B D C) 4) (A B D C) 8)

In [115]:
; 符号化処理
; 回答
(define (encode-symbol symbol tree)
  (define (iter sub result)
    (if (leaf? sub)
        (if (eq? (symbol-leaf sub) symbol) result
            '())
        (let ((l (left-branch sub))
              (r (right-branch sub)))
          (let ((l-result (iter l (append result '(0)))))
            (if (not (null? l-result)) l-result
                (iter r (append result '(1)))
              )
            )
        )
      )
    )
  (let ((result (iter tree '())))
    (if (null? result) (error "bad symbol: ENCODE" symbol)
        result
        )
    )
)

In [116]:
; 個々の記号で動作確認
(display (encode-symbol 'A sample-tree))
(newline)
(display (encode-symbol 'B sample-tree))
(newline)
(display (encode-symbol 'C sample-tree))
(newline)
(display (encode-symbol 'D sample-tree))
(newline)
;(display (encode-symbol 'E sample-tree))
;(newline)

(0)
(1 0)
(1 1 1)
(1 1 0)


In [117]:
; 問題の符号化処理 
(define (encode message tree)
    (if (null? message) '()
        (append
            (encode-symbol (car message) tree)
            (encode (cdr message) tree))
    )
)

In [118]:
(encode '(A D A B B C A) sample-tree)

(0 1 1 0 0 1 0 1 0 1 1 1 0)

In [119]:
(define sample2 (list (list 'A 4) (list 'B 2) (list 'D 1) (list 'C 1)))
sample2

((A 4) (B 2) (D 1) (C 1))

In [120]:
; 逆順で返ってくることに注意
(make-leaf-set sample2)

((leaf C 1) (leaf D 1) (leaf B 2) (leaf A 4))

In [121]:
; 間違った実装
(define (successive-merge set)
  (define (iter sub)
    (cond ((null? sub) '())
          ((= (length sub) 1) sub)
          ((= (length sub) 2) (make-code-tree (car sub) (cadr sub)))
          (else (make-code-tree (car sub) (iter (cdr sub))))
        )
    )
  (iter (reverse set))
  )

In [122]:
(define (generate-huffman-tree pairs)
    (successive-merge (make-leaf-set pairs)))

In [123]:
(define sample-tree2 (generate-huffman-tree sample2))
sample-tree2

((leaf A 4) ((leaf B 2) ((leaf D 1) (leaf C 1) (D C) 2) (B D C) 4) (A B D C) 8)

In [239]:
;
(define (successive-merge set)
  (define (iter sub result)
    (cond ((null? sub) result)
          ((null? result) (iter (cddr sub) (make-code-tree (cadr sub) (car sub))))
          ;((not (pair? sub))
          ; (if (>= (weight sub) (weight result)) (make-code-tree sub result)
          ;     (make-code-tree result sub)
          ;     )
          ; )
          ;((= (length sub) 1) (make-code-tree sub result))
          ((>= (length sub) 3) (iter (cddr sub) (make-code-tree (make-code-tree (cadr sub) (car sub)) result)))
          (else
           (let ((top (car sub))
                 (next (cdr sub)))
             (display sub)
             (newline)
             (display top)
             (newline)
             (display next)
             (newline)
             (display (weight top))
             (newline)
             (display (weight result))
             (newline)
             ;(if (>= (weight top) (weight result)) (iter next (make-code-tree top result))
             ;    ;(make-code-tree (iter next top) result)
             ;    ;(iter next (make-code-tree result top))
             ;    (iter next (make-code-tree top result))
             ;  )
             (iter next (make-code-tree top result))
             )
           )
        )
    )
  (iter set '())
  )

In [240]:
(define sample-tree2 (generate-huffman-tree sample2))
sample-tree2

((leaf B 2) (leaf A 4))
(leaf B 2)
((leaf A 4))
2
2
((leaf A 4))
(leaf A 4)
()
4
4


((leaf A 4) ((leaf B 2) ((leaf D 1) (leaf C 1) (D C) 2) (B D C) 4) (A B D C) 8)

In [241]:
(define sample-tree3 
  (generate-huffman-tree
     (list (list 'A 8) (list 'B 3) (list 'C 1) (list 'D 1) (list 'E 1) (list 'F 1) (list 'G 1) (list 'H 1)))
  )
sample-tree3

((leaf B 3) (leaf A 8))
(leaf B 3)
((leaf A 8))
3
6
((leaf A 8))
(leaf A 8)
()
8
9


((leaf A 8) ((leaf B 3) (((leaf C 1) (leaf D 1) (C D) 2) (((leaf E 1) (leaf F 1) (E F) 2) ((leaf G 1) (leaf H 1) (G H) 2) (E F G H) 4) (C D E F G H) 6) (B C D E F G H) 9) (A B C D E F G H) 17)

In [242]:
(display (encode '(A) sample-tree2))
(newline)
(display (encode '(B) sample-tree2))
(newline)
(display (encode '(C) sample-tree2))
(newline)
(display (encode '(D) sample-tree2))
(newline)


(0)
(1 0)
(1 1 1)
(1 1 0)


In [243]:
(display (encode '(A) sample-tree3))
(newline)
(display (encode '(B) sample-tree3))
(newline)
(display (encode '(C) sample-tree3))
(newline)
(display (encode '(D) sample-tree3))
(newline)
(display (encode '(E) sample-tree3))
(newline)
(display (encode '(F) sample-tree3))
(newline)
(display (encode '(G) sample-tree3))
(newline)
(display (encode '(H) sample-tree3))
(newline)

(0)
(1 0)
(1 1 0 0)
(1 1 0 1)
(1 1 1 0 0)
(1 1 1 0 1)
(1 1 1 1 0)
(1 1 1 1 1)


In [196]:
(decode '(0 1 1 0 0 1 0 1 0 1 1 1 0) sample-tree2)

(A D A B B C A)

In [197]:
(encode '(A D A B B C A) sample-tree2)

(0 1 1 0 0 1 0 1 0 1 1 1 0)

In [199]:
(decode '(1 0 0 1 1 0 0 1 1 1 0 0 1 1 1 1 0 0 1 1 1 1 1 0 0 1 0 1 0 0 0 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1) sample-tree3)

(A C A C A A C A A A C A A A A C B E A A A A A C A A A A A A)

In [200]:
(decode '(1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 1 0 0 0 1 1 0 1 0 1 0 0 1 0 0 0 0 0 1 1 1 0 0 1 1 1 1) sample-tree3)

(A D B C B A B A D A B B C F A A C A A A)

In [201]:
(encode '(B A C A D A E A F A B B A A A G A H) sample-tree3)

(0 1 1 0 0 1 1 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 0 1 1 0 1 0 1 1 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1)

In [99]:
(define sample-tree4
    (make-code-tree (make-leaf 'A 8)
        (make-code-tree
            (make-code-tree (make-leaf 'B 3)
                            (make-code-tree (make-leaf 'C 1) (make-leaf 'D 1))
            )
            (make-code-tree (make-code-tree (make-leaf 'E 1) (make-leaf 'F 1))
                            (make-code-tree (make-leaf 'G 1) (make-leaf 'H 1))
            )
        )
    )
)
sample-tree4

((leaf A 8) (((leaf B 3) ((leaf C 1) (leaf D 1) (C D) 2) (B C D) 5) (((leaf E 1) (leaf F 1) (E F) 2) ((leaf G 1) (leaf H 1) (G H) 2) (E F G H) 4) (B C D E F G H) 9) (A B C D E F G H) 17)

In [100]:
(display (encode '(A) sample-tree4))
(newline)
(display (encode '(B) sample-tree4))
(newline)
(display (encode '(C) sample-tree4))
(newline)
(display (encode '(D) sample-tree4))
(newline)
(display (encode '(E) sample-tree4))
(newline)
(display (encode '(F) sample-tree4))
(newline)
(display (encode '(G) sample-tree4))
(newline)
(display (encode '(H) sample-tree4))
(newline)

(0)
(1 0 0)
(1 0 1 0)
(1 0 1 1)
(1 1 0 0)
(1 1 0 1)
(1 1 1 0)
(1 1 1 1)


In [101]:
(decode '(1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 1 0 0 0 1 1 0 1 0 1 0 0 1 0 0 0 0 0 1 1 1 0 0 1 1 1 1) sample-tree4)

(B A C A D A E A F A B B A A A G A H)

In [102]:
(encode '(B A C A D A E A F A B B A A A G A H) sample-tree4)

(1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 1 0 0 0 1 1 0 1 0 1 0 0 1 0 0 0 0 0 1 1 1 0 0 1 1 1 1)

In [103]:
(cddr '(1 2 3 4))

(3 4)